# Email Data Pipeline

This notebook extracts data from qardio google sheets, cleans the data, and loads it into the Qardio SQL database

In [1]:
from google_apis import gsheets_functions as gs
import pandas as pd
import sqlalchemy

database = 'qardio'
host = '127.0.0.1'
user = 'root'
password = 'Party100'
url = f'mysql+mysqlconnector://{user}:{password}@{host}/{database}'
engine = sqlalchemy.create_engine(url, echo=True)
conn = engine.connect()

pd.set_option('display.max_columns', 500)

2023-06-22 12:56:20,175 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-06-22 12:56:20,176 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-06-22 12:56:20,178 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-06-22 12:56:20,179 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-06-22 12:56:20,180 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-06-22 12:56:20,181 INFO sqlalchemy.engine.Engine [raw sql] {}


In [2]:
qardio_data = gs.gspread_read(wb='1dKiIp2ETyzfKHzR9fMeHrgf2N_UKcSa5uPWathJzgIw',
                              ws='22_+_23')

Clean column headers

In [3]:
qardio_data.columns = qardio_data.columns.str.strip()
qardio_data.columns = qardio_data.columns.str.lower()
qardio_data.columns = qardio_data.columns.str.replace(' ', '_')
qardio_data.columns = qardio_data.columns.str.replace('.', '')

Solve dtypes

In [4]:
qardio_data.date_sent = pd.to_datetime(qardio_data.date_sent)
for col in qardio_data.loc[:,'recipients':].select_dtypes('object').columns:
    qardio_data[col] = qardio_data[col].str.replace('$', '')
    qardio_data[col] = qardio_data[col].str.replace('%', '')
    qardio_data[col] = qardio_data[col].str.replace(',', '')
    qardio_data[col] = qardio_data[col].str.replace('#DIV/0!', '')
    qardio_data[col] = pd.to_numeric(qardio_data[col])

In [5]:
qardio_data.dtypes

campaign_name                     object
year                               int64
date_sent                 datetime64[ns]
day                               object
time_pst                          object
subject_line                      object
subscribers                       object
country                           object
recipients                         int64
opens                              int64
open_rate                        float64
clicks                           float64
click_rate                       float64
true_click_rate                  float64
click_per_unique_opens           float64
unsubs                             int64
%_unsubs                         float64
%_unsubs_openers                 float64
bounced                            int64
sessions                         float64
sess_per_unique_opens            float64
ecr                              float64
total_trans                      float64
revenue                          float64
aov             

## Save data to SQL database

In [6]:
qardio_data.to_sql(name='email_data',
          con=conn,
          if_exists='replace')

2023-06-22 12:56:22,382 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-22 12:56:22,405 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2023-06-22 12:56:22,406 INFO sqlalchemy.engine.Engine [generated in 0.00133s] {'table_schema': 'qardio', 'table_name': 'email_data'}
2023-06-22 12:56:22,413 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2023-06-22 12:56:22,415 INFO sqlalchemy.engine.Engine [cached since 0.009928s ago] {'table_schema': 'qardio', 'table_name': 'email_data'}
2023-06-22 12:56:22,417 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `qardio`
2023-06-22 12:56:22,419 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-06-22 12:56:22,438 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `email_data`
2023-06-22 12:56:22,440 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-06-22 1